In [1]:
#Import packages
import pandas as pd
import numpy as np
import glob
import re
import string
import codecs

In [12]:
#Create single parliament_qs dataframe with all question data
speech = []




for file in glob.glob('./Modi_Speeches/*.txt'):
    new = open(file, 'r', encoding="ascii", errors="surrogateescape").read().replace("/n" ,"")
    speech.append(new)

speech
# parliament_qs = pd.concat(li, axis = 0, ignore_index = True)

['\udcef\udcbb\udcbfMy Dear Countrymen,\nNamaskar!\nI was not feeling like doing Mann ki Baat today. I am feeling heavy and burdened and little distressed. There were news of hailstorm, unseasonal rains and damages to farmer\udce2\udc80\udc99s crop when I was talking to all of you last month. Suddenly, out of nowhere strong winds blew in Bihar, causing death of so many people and inflicted lot of damages there. And the earthquake has shaken the entire world on Saturday. It seems the trend of natural disasters has continued. Be it the Nepal earthquake and in India as well the earthquake took lives across various states of India. There has been loss of property because of the earthquake. However, the loss to Nepal is frightening.\nI have seen Kutch earthquake very closely on Jan 26, 2001. I can clearly imagine how devastating these disasters are. I can imagine what Nepal and the victimized families there must be going through. But my dear brothers and sisters of Nepal, India is with you 

In [13]:
#Code borrowed and adapted from George Chen, Carnegie Mellon University#
#Define function to remove punctuation and whitespace, and lowercase all text
def makeWordList(str_object):
    
    corpus_text = str(str_object)
    
    for c in string.punctuation:
        corpus_text = corpus_text.replace(c, "")  # -- (1)
    
    text = re.sub(r'\S*\d\S*','',corpus_text) # -- (2)
    text = re.sub(r'[^\w\s]','',text)         # -- (3)
    
    text = text.lower().split()           # -- (4)         
    
    li = []
    for token in text:
        li.append(token)

    return " ".join(li)

In [14]:
#Process the questions
processed_speech = []

for str_object in speech:
    processed_speech.append(makeWordList(str_object))

In [18]:
#Use CountVectorizer to transform parliamentary questions
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(min_df=2, stop_words="english", max_df=0.8)
speech_fit = vectorizer.fit(processed_speech)
X_speech = vectorizer.fit_transform(processed_speech).toarray()

In [19]:
#!Time-consuming!#
#Create topics using LDA
num_topics = 10

from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=num_topics, learning_method='online', random_state=0)
lda.fit(X_speech)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='online', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=10, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [20]:
#Display top 10 words from each topic
words = list(speech_fit.vocabulary_)
topic_word_distributions = np.array([row / row.sum() for row in lda.components_])
num_top_words = 10

print('Displaying the top 10 words per topic and their probabilities within the topic...')
print()

for topic_idx in range(num_topics):
    print('[Topic ', topic_idx, ']', sep='')
    sort_indices = np.argsort(-topic_word_distributions[topic_idx])
    for rank in range(num_top_words):
        word_idx = sort_indices[rank]
        print(words[word_idx], ':', topic_word_distributions[topic_idx, word_idx])
    print()

Displaying the top 10 words per topic and their probabilities within the topic...

[Topic 0]
intelligent : 0.005115790153545838
rates : 0.0049437430265327435
animals : 0.004904734149356382
deserves : 0.004567552648969746
gram : 0.004277229571516617
disappointment : 0.003995182158183563
credit : 0.003857352550608951
monsoons : 0.0038107949862773017
karun : 0.0031455615056786043
issue : 0.0027073091851165278

[Topic 1]
academic : 0.0006977373484135768
mahamana : 0.0006816046455224806
chemicals : 0.0006769408318717812
chief : 0.0006627852467680016
watching : 0.0006097585680707229
designed : 0.0006021900726136001
pious : 0.0006014008094708439
politics : 0.0005906563493936473
based : 0.0005812735724478874
infuse : 0.000580828598468114

[Topic 2]
narain : 0.001290547000881144
pious : 0.0012293623299197557
politics : 0.0007136834987456179
eager : 0.0006866890701943369
educated : 0.0006696532724943853
chief : 0.0006624188627768151
watching : 0.000639402645234237
fresh : 0.0006370568550587311
d

In [ ]:
len(processed_questions)

In [ ]:
X_questions